# Open Archive

Open a file/object archive into a target directory

### Parameters

* **`context`**: `mlrun.MLClientCtx` - function execution context
* **`archive_url`**: `str` - url of archive file
* **`subdir`**: `str` - path within artifact store where extracted files are stored
* **`key`**: `str` - key of archive contents in artifact store
* **`target_path`**: `str` - file system path to store extracted files

### Usage

In [10]:
import mlrun
import os

project_name = "function-marketplace"
project = mlrun.get_or_create_project(project_name, context="./", user_project=False)

> 2022-11-24 09:53:14,477 [info] loaded project function-marketplace from MLRun DB


In [ ]:
# Declaring our zip target url
ARCHIVE_URL = "https://s3.wasabisys.com/iguazio/data/cats-vs-dogs/cats-vs-dogs-labeling-demo.zip"
TARGET_PATH = os.path.join(os.getcwd(),'content/')

# Import, mount & run
fn = mlrun.import_function("hub://open_archive")

fn.apply(mlrun.auto_mount())
if os.getenv('MLRUN_CE','False')=='False':
    fn.spec.disable_auto_mount=False
    
run = fn.run(inputs={'archive_url': ARCHIVE_URL},
             params={'key': 'test_archive',
                     'target_path': TARGET_PATH})

### Inspecting outputs

In [ ]:
print("Artifact URL : ", run.artifact('test_archive').url)

# getting folder size
size = 0
# get size
for path, dirs, files in os.walk(TARGET_PATH):
    for f in files:
        fp = os.path.join(path, f)
        size += os.stat(fp).st_size
        
print("Extracted folder size : ", size)

# Deleting folder
import shutil
shutil.rmtree(TARGET_PATH + '/data')